In [ ]:
import json
import pathlib 
import requests 
import pandas as pd
from urllib.parse import urlencode
import pathlib  # módulo para manejar paths de forma portable (Windows/macOS/Linux)

In [ ]:
# Endpoint base del servicio OData v2 (Northwind público)
BASE_URL = "https://services.odata.org/V2/Northwind/Northwind.svc"

# Carpeta de caché local. para no descargar siempre de la red

CACHE = pathlib.Path("data_cache")          # crea un objeto "Path" que apunta a ./data_cache
CACHE.mkdir(parents=True, exist_ok=True)    # crea la carpeta si no existe (parents=True por si faltan carpetas intermedias)

In [ ]:
def odata_to_df(entity: str, 
                select: str | None = None, 
                page_size: int = 500, 
                use_cache: bool = True) -> pd.DataFrame:
    """
    Descarga TODOS los registros de una entidad OData v2 (Northwind) y los devuelve como DataFrame.
    - entity: nombre de la colección (p. ej., "Orders", "Customers", "Order_Details").
    - select: columnas opcionales separadas por coma para $select (reduce ancho de respuesta).
    - page_size: tamaño de página para paginación ($top / $skip).
    - use_cache: si True, guarda/lee un JSON local para acelerar iteraciones.

    Implementa paginación v2:
      - pide $top=page_size y va incrementando $skip hasta que la página devuelta tenga < page_size filas.
    Estructura JSON v2 esperada: {"d": {"results": [ ...filas... ]}}
    """
    # Archivo de caché local (por entidad)
    cache_file = CACHE / f"{entity}.json"

    # Si existe caché y se permite, leemos desde disco (sin pegarle al endpoint)
    if use_cache and cache_file.exists():
        rows = json.loads(cache_file.read_text(encoding="utf=-8"))
        return pd.DataFrame(rows)

    # Armamos los parámetros base de OData
    params = {"$format" : "json"} # pedimos JSON (v2 puede dar XML si no se aclara)
    if select:
        params ["$select"] = select

    # Paginación con $top/$skip
    rows, skip = [], 0
    while True:
        # Unimos params "base" con la paginación para esta página
        q = params | {"$top": page_size, "$skip": skip}
        # Construimos la URL final: BASE_URL/<entity>?<querystring>
        url = f"{BASE_URL}/{entity}?{urlencode(q)}"

        # Llamada HTTP GET con timeout (evita quedarte colgado)
        r = requests.get(url, timeout=60)
        # Si algo salió mal (HTTP 4xx/5xx), levanta excepción con detalle
        r.raise_for_status()

        # OData V2 (Northwind) envuelve resultados en d.results
        page = r.json().get("d", {}).get("results", [])
        rows.extend(page)

        # Condición de fin: si la página trae menos de page_size filas, no hay más
        if len(page) < page_size:
            break


        # Preparamos siguiente iteración: saltamos lo ya leído
        skip += page_size

    # Si hay que cachear, guardamos JSON a disco para acelerar próximos análisis
    if use_cache:
        cache_file.write_text(json.dumps(rows), encoding="utf-8")

    # Devolvemos DataFrame listo para tu EDA
    return pd.DataFrame(rows)